This notebook contains examples of using model index and OPC UA functions to download inverters, strings set and trackers data of the first site from the model index and OPC UA api servers. The aggregated historical data downloaded in the data folder and cell execution time can be noted for each request.

### Import Libraries

In [ ]:
# Import the required packeages
import pandas as pd
from typing import List, Dict
import sys
import os
import json
import asyncio
import datetime
from aiohttp import ClientSession

### Import Scripts

In [ ]:
# Setting the path
module_path = os.path.abspath(os.path.join("../src/pyprediktormapclient/"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# Import model index functions
from pyprediktormapclient.model_index import ModelIndex

# Import OPC UA functions
from pyprediktormapclient.opc_ua import OPC_UA

# Import Analytics Helper
from pyprediktormapclient.analytics_helper import AnalyticsHelper

# Import "Dataframer" Tools
from pyprediktormapclient.shared import *

In [ ]:
# Connection to the servers
model_index_url = "http://uidev-ubuntu-02.prediktor.no:13701/v1/"

# Model index API
model = ModelIndex(url=model_index_url)

### OPC UA server parameters

In [ ]:
model.get_objects_of_type("InverterType")

In [ ]:
sites_json = model.get_objects_of_type("SiteType")
sites_json

In [ ]:
# All the sites on the OPC server
sites_json = model.get_objects_of_type("SiteType")
sites = AnalyticsHelper(sites_json)
sites.dataframe

In [ ]:
# Selecting the second site
all_site_ids = sites.list_of_ids()
first_site_id = all_site_ids[1]
first_site_id

In [ ]:
# Initate the OPC UA connection
opcua_rest_url = "http://uidev-ubuntu-02.prediktor.no:13702/"
opcua_server_url = "opc.tcp://10.100.59.154:4860"

tsdata = OPC_UA(rest_url=opcua_rest_url, opcua_url=opcua_server_url, namespaces=sites.namespaces_as_list(model.get_namespace_array()))

INVERTERS DATA

In [ ]:
# All the inverters on the site
inverters_json = model.get_object_descendants("InverterType", all_site_ids, "PV_Assets")
inverters = AnalyticsHelper(inverters_json)
inverters.dataframe

In [ ]:
# List of unique inverter variables
inv_var_list = [
    "ACActivePower",
    "ACCurrentAverage",
    "ACCurrentPhaseA",
    "ACCurrentPhaseB",
    "ACCurrentPhaseC",
    "ACDailyActiveEnergy",
    "ACReactivePower",
    "ACTotalActiveEnergy",
    "ACVoltagePhaseAB",
    "ACVoltagePhaseAC",
    "ACVoltagePhaseBC",
    "DCCurrent",
    "DCPower",
    "DCVoltage",
    "Frequency",
    "PowerFactor",
    "Status",
]

In [ ]:
# Live value inverters data
inv_liv_values = tsdata.get_values(
    inverters.variables_as_list(inv_var_list)
)
inv_liv_values

In [ ]:
# 1 day aggregated historical inverter data
one_day_historic_inverter_data = tsdata.get_historical_aggregated_values(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=60*1000,
    agg_name="Average",
    variable_list=inverters.variables_as_list(inv_var_list),
)
one_day_historic_inverter_data

In [ ]:
# 1 day aggregated historical inverter data in asyncio process
one_days_historic_inverter_data2 = await tsdata.get_historical_aggregated_values_async(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=60*1000,
    agg_name="Average",
    variable_list=inverters.variables_as_list(inv_var_list),
    batch_size=500
)
one_days_historic_inverter_data2